# Обучение модели FLAN-T5

In [1]:
#%pip install --upgrade pip

In [2]:
#%pip install -q -U einops datasets matplotlib tqdm boto3 git+https://github.com/dask/s3fs

In [3]:
#%pip install wandb

In [4]:
#%pip install transformers

In [5]:
#%pip install evaluate

In [6]:
#%pip install accelerate -U

In [7]:
#%pip install rouge-score

In [8]:
#%pip install bert_score

In [9]:
import pandas as pd
import os
from utils import scripts_rework
import warnings

warnings.filterwarnings("ignore")

In [10]:
df = pd.read_pickle("data/scripts_reworked.pkl")
df

,answer,question,context
0,"There’s no point, I just think it’s a good id...","Agreed, what’s your point?",
1,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","Hang on. One across is Aegean, eight down is..."
2,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","One across is Aegean, eight down is Nabakov, ..."
3,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","Can I help you? Yes. Um, is this the High IQ ..."
4,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","Yes. Um, is this the High IQ sperm bank?"
...,...,...,...
48859,"Well, that would raise a number of problems. ...",What if I were?,"Well, perfect. I made us sandwiches. How thou..."
48860,"Well, that would raise a number of problems. ...",What if I were?,"How thoughtful. Thank you. Mmm. No big deal, ..."
48861,"Well, that would raise a number of problems. ...",What if I were?,"Mmm. No big deal, I enjoy spending time with ..."
48862,"Well, that would raise a number of problems. ...",What if I were?,"And I with you. Question, are you seeking a r..."


In [11]:
df["combined"] = df[["question", "context"]].apply(
    lambda df: "context: " + df["context"] + "</s>" + 'question: '
+ df['question'], axis = 1)
df

,answer,question,context,combined
0,"There’s no point, I just think it’s a good id...","Agreed, what’s your point?",,"context: </s>question: Agreed, what’s your po..."
1,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","Hang on. One across is Aegean, eight down is...","context: Hang on. One across is Aegean, eigh..."
2,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","One across is Aegean, eight down is Nabakov, ...","context: One across is Aegean, eight down is ..."
3,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","Can I help you? Yes. Um, is this the High IQ ...","context: Can I help you? Yes. Um, is this the..."
4,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","Yes. Um, is this the High IQ sperm bank?","context: Yes. Um, is this the High IQ sperm b..."
...,...,...,...,...
48859,"Well, that would raise a number of problems. ...",What if I were?,"Well, perfect. I made us sandwiches. How thou...","context: Well, perfect. I made us sandwiches...."
48860,"Well, that would raise a number of problems. ...",What if I were?,"How thoughtful. Thank you. Mmm. No big deal, ...",context: How thoughtful. Thank you. Mmm. No b...
48861,"Well, that would raise a number of problems. ...",What if I were?,"Mmm. No big deal, I enjoy spending time with ...","context: Mmm. No big deal, I enjoy spending t..."
48862,"Well, that would raise a number of problems. ...",What if I were?,"And I with you. Question, are you seeking a r...","context: And I with you. Question, are you se..."


In [12]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.01, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
train_df.shape, val_df.shape

((48375, 4), (489, 4))

In [13]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
        "valid": Dataset.from_pandas(val_df.reset_index(drop=True)),
    }
)
dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context', 'combined'],
        num_rows: 48375
    })
    valid: Dataset({
        features: ['answer', 'question', 'context', 'combined'],
        num_rows: 489
    })
})

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [15]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["valid"]]).map(
    lambda x: tokenizer(x["combined"], truncation=True),
    batched=True,
    remove_columns=["context", "question", "answer", "combined"],
)
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["valid"]]).map(
    lambda x: tokenizer(x["answer"], truncation=True),
    batched=True,
    remove_columns=["context", "question", "answer", "combined"],
)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Parameter 'function'=<function <lambda> at 0x7fd1cffb6b00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/48864 [00:00<?, ? examples/s]

Max source length: 332


Map:   0%|          | 0/48864 [00:00<?, ? examples/s]

Max target length: 304


In [16]:
def preprocess_function(sample, padding="max_length"):

    model_inputs = tokenizer(
        sample['combined'], max_length=max_source_length, padding=padding, truncation=True
    )

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(
        text_target=sample["answer"],
        max_length=max_target_length,
        padding=padding,
        truncation=True,
    )

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=["context", "question", "answer", "combined"]
)
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/48375 [00:00<?, ? examples/s]

Map:   0%|          | 0/489 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [17]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id = "google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [18]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

# Metric
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")


# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    bertscore_result = bertscore.compute(predictions=decoded_preds, references=decoded_labels,lang="en")
    

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    rouge_result = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    rouge_result = {k: round(v * 100, 4) for k, v in rouge_result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    rouge_result["gen_len"] = np.mean(prediction_lens)
    
    result = {'rouge_1': round(rouge_result['rouge1'], 4),
              'rouge_2': round(rouge_result['rouge2'], 4),
              'rouge_L': round(rouge_result['rougeL'], 4), 
              'avg_len': round(rouge_result['gen_len'], 4), 
              'bertscore_prec': round(np.mean(bertscore_result['precision']), 4),
              'bertscore_rec': round(np.mean(bertscore_result['recall']), 4),
              'bertscore_f1': round(np.mean(bertscore_result['f1']), 4)}
    return result

2024-03-07 10:05:58.360094: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, label_pad_token_id=label_pad_token_id, pad_to_multiple_of=8
)

In [20]:
from huggingface_hub import HfFolder
HfFolder.save_token(os.environ['hugging_face_login'])

In [21]:
import wandb
os.environ["WANDB_PROJECT"] = "generative_models_chat"
os.environ["WANDB_LOG_MODEL"] = "true"
wandb.login(key=os.environ["wandb_login"])

wandb: Currently logged in as: katya_shakhova (shakhova). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


True

In [22]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-sheldon-chat-v2"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    warmup_ratio=0.1,
    weight_decay=0.1,
    optim="adamw_torch",
    # logging & evaluation strategies
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="no",
    save_total_limit=1,
    report_to="wandb",
    logging_steps=200,
    push_to_hub=False,
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    compute_metrics=compute_metrics,
)

In [23]:
trainer.train()

wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.3
wandb: Run data is saved locally in /home/jupyter/work/resources/chat_bot_katya_HW2/wandb/run-20240307_100606-t5nq3oct
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run celestial-hill-37
wandb: ⭐️ View project at https://wandb.ai/shakhova/generative_models_chat
wandb: 🚀 View run at https://wandb.ai/shakhova/generative_models_chat/runs/t5nq3oct


Step,Training Loss,Validation Loss,Rouge 1,Rouge 2,Rouge L,Avg Len,Bertscore Prec,Bertscore Rec,Bertscore F1
200,3.573100,3.255625,9.288600,1.782600,8.855800,10.881400,0.866100,0.849400,0.857300
400,3.421000,3.175784,9.255900,1.906400,8.824300,10.707600,0.871400,0.853800,0.862200
600,3.345700,3.119731,9.099100,2.185400,8.707600,10.106300,0.874100,0.854600,0.863900
800,3.301300,3.075485,8.552300,1.889600,8.222400,12.368100,0.870600,0.853700,0.861700
1000,3.250900,3.038346,9.239600,2.040900,8.831400,12.051100,0.872900,0.855200,0.863600
1200,3.237600,3.000550,8.735600,1.818100,8.303900,12.979600,0.870200,0.854100,0.861700
1400,3.207500,2.960210,9.430300,2.000300,8.757100,16.073600,0.863300,0.854800,0.858700
1600,3.134200,2.926318,10.814800,2.269800,9.873500,15.482600,0.866000,0.857100,0.861100
1800,3.137600,2.891541,9.938100,1.856900,9.129100,15.171800,0.863500,0.855300,0.859000
2000,3.086200,2.864813,9.653400,1.792500,8.953900,14.993900,0.860300,0.853800,0.856700


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TrainOutput(global_step=15120, training_loss=2.696330834444238, metrics={'train_runtime': 10620.0465, 'train_samples_per_second': 22.775, 'train_steps_per_second': 1.424, 'total_flos': 1.0869185553408e+17, 'train_loss': 2.696330834444238, 'epoch': 5.0})

In [24]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Shakhovak/flan-t5-base-sheldon-chat-v2/commit/ae2a7ee107d9da041aa3be37333ec32b3e9d9f5a', commit_message='End of training', commit_description='', oid='ae2a7ee107d9da041aa3be37333ec32b3e9d9f5a', pr_url=None, pr_revision=None, pr_num=None)

In [25]:
wandb.finish()

wandb: ERROR Control-C detected -- Run data was not synced
